<a href="https://colab.research.google.com/github/tugcebyrl/Telegram-AI-BOT/blob/main/Telegram_AI_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install python-telegram-bot==13.7

In [ ]:
#!pip install tensorflow

# Import Modules
* token to access the telegram bot api

In [ ]:
from telegram.ext import *
from io import BytesIO
import cv2
import numpy as np
import tensorflow as tf

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/token.txt','r') as f:
  telegram_token=f.read()

# Training Data

In [ ]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()
x_train,x_test=x_train/255,x_test/255

In [ ]:
class_names=['Plane','Car','Cat','Horse','Deer','Rose','Frog','Ship','Mouse','Truck']

# Neural Network

In [ ]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(32,32,3)))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='softmax'))

# Basic Bot Settings

In [ ]:
def start(update,context):
  update.message.reply_text("HELLO!")

In [ ]:
def help(update,context):
  update.message.reply_text("""
  /start - Starts Conversation
  /help - Messages
  /train - Trains Network
  """)

# Process images from users

In [ ]:
def train(update,context):
  update.message.reply_text("... Model is being trained ...")
  model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))
  #model.save('cifar_classifier.model')
  model.export('cifar_classifier')
  update.message.reply_text("... Process Done! You can send pictures  ...")

In [ ]:
def handle_message(update,context):
  update.message.reply_text("Send image to train model")

In [ ]:
def handle_photo(update,context):
  file=context.bot.get_file(update.message.photo[-1].file_id)
  f=BytesIO(file.download_as_bytearray())
  file_bytes=np.asarray(bytearray(f.read()),dtype=np.uint8)

  img=cv2.imdecode(file_bytes,cv2.IMREAD_COLOR)
  img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
  img=cv2.resize(img,(32,32),interpolation=cv2.INTER_AREA)

  pred=model.predict(np.array([img/255]))
  update.message.reply_text(f"I think the picture is a {class_names[np.argmax(pred)]}")

In [ ]:
updater = Updater(telegram_token,use_context=True)
dp=updater.dispatcher

In [ ]:
dp.add_handler(CommandHandler("start",start))
dp.add_handler(CommandHandler("help",help))
dp.add_handler(CommandHandler("train",train))
dp.add_handler(MessageHandler(Filters.text,handle_message))
dp.add_handler(MessageHandler(Filters.photo,handle_photo))

In [ ]:
updater.start_polling()
updater.idle()

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 94s 59ms/step - accuracy: 0.3465 - loss: 1.7548 - val_accuracy: 0.5520 - val_loss: 1.2383
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 130s 51ms/step - accuracy: 0.5757 - loss: 1.2013 - val_accuracy: 0.5988 - val_loss: 1.1182
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 91s 58ms/step - accuracy: 0.6433 - loss: 1.0206 - val_accuracy: 0.6512 - val_loss: 0.9825
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 131s 51ms/step - accuracy: 0.6797 - loss: 0.9152 - val_accuracy: 0.6722 - val_loss: 0.9307
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 80s 51ms/step - accuracy: 0.7087 - loss: 0.8277 - val_accuracy: 0.6883 - val_loss: 0.8990
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 83s 52ms/step - accuracy: 0.7299 - loss: 0.7690 - val_accuracy: 0.6917 - val_loss: 0.8908
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 79s 50ms/step - accuracy: 0.7500 - loss: 0.7110 - val_accuracy: 0.6776 - val_loss: 0.9678
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 83s 51ms/step - accuracy: 0.7643